# An approach to identifying the best customers

Different businesses define best customers according to their business goals.

For example, 
* a store has a total budget of 1000$ for discount coupons 
* must come up with an approach to identify their best customers during Christmas
* the discount coupons will be distributed equally out of the budget

The data set consists of a few thousand customer transactions over time.

In [13]:
import pandas as pd
import datetime as dt

In [14]:
#importing customer data with transaction dates and amounts
data = pd.read_csv("rfm_xmas19.txt", parse_dates=["trans_date"])
#parse_dates allows us to immediately set the appropriate date type to the selected columns
data.info()
#no missing values

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125000 entries, 0 to 124999
Data columns (total 3 columns):
customer_id    125000 non-null object
trans_date     125000 non-null datetime64[ns]
tran_amount    125000 non-null int64
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 2.9+ MB


# Understanding Customer Churn
In the case of subscription-based businesses, **customer churn** is an event when a customer unsubscribes. In this example, we define churned customers as those who have not made any transactions in the last three months.

We will remove these churned customers and use only the remaining ones for our final goal of identifying the best customers.

In [15]:
group_by_customer = data.groupby("customer_id")

#exploring the last transactions of customers
last_transaction = group_by_customer["trans_date"].max()
last_transaction.head()

customer_id
FM1112   2019-10-14
FM1113   2019-11-09
FM1114   2019-11-12
FM1115   2019-12-05
FM1116   2019-05-25
Name: trans_date, dtype: datetime64[ns]

In [16]:
#converting into dataframe 
best_churn = pd.DataFrame(last_transaction)

#defining the cut off date as a date three months ago
cutoff_day = dt.datetime(2019, 10, 16)

#filtering out churned customers by assigning them a value of 0 if they havent had a transaction since cutoff_day
best_churn["churned"] = best_churn["trans_date"].apply(
    lambda date: 1 if date < cutoff_day else 0)

#examining the new column
best_churn["churned"].head()

customer_id
FM1112    1
FM1113    0
FM1114    0
FM1115    0
FM1116    1
Name: churned, dtype: int64

# Ranking Mechanism

Now that we have customers who have been active in the last three months, we need to design a **ranking mechanism**. 

For this exercise:
* We will use a very simple weighted sum model to classify customers. 
* A score represents how valuable a customer is.

Our formula to define the score of a customer is

<blockquote> (1/2 * Number of purchases) + (1/2 * Total amount spent)</blockquote>

 
We're going to use the groupby object group_by_customer we created to get the number of purchases and the total amount spent by each customer — the necessary components for this score.

In [17]:
best_churn["nr_of_transactions"] = group_by_customer.size()
best_churn["amount_spent"] = group_by_customer.sum()
#dropping trans_date since we have already used it to get our customers
best_churn.drop("trans_date", axis="columns", inplace=True)

# Feature Scaling 
While the above ranking mechanism takes both factors into consideration, the amount_spent feature has more weightage due to the amount in dollars (49-2933) being on a different scale than the nr_of_transactions (4-39).

It is important to normalize the features into a single scale so that we can use both comparatively to define our metric.

In [18]:
#scaling the nr_of_transactions
best_churn["scaled_tran"] = (best_churn["nr_of_transactions"] \
                             - best_churn["nr_of_transactions"].min()) \
                             / (best_churn["nr_of_transactions"].max() \
                             - best_churn["nr_of_transactions"].min())

#scaling the amount_spent
best_churn["scaled_amount"] = (best_churn["amount_spent"] \
                               -best_churn["amount_spent"].min()) \
                               / (best_churn["amount_spent"].max() \
                               - best_churn["amount_spent"].min())

#defining our score with the formula above
best_churn["score"] = 100*(.5*best_churn["scaled_tran"] \
                           + .5*best_churn["scaled_amount"])

#checking out the score column
best_churn.head()

,churned,nr_of_transactions,amount_spent,scaled_tran,scaled_amount,score
customer_id,,,,,,
FM1112,1,15,1012,0.314286,0.309986,31.213567
FM1113,0,20,1490,0.457143,0.481681,46.941195
FM1114,0,19,1432,0.428571,0.460848,44.470956
FM1115,0,22,1659,0.514286,0.542385,52.833539
FM1116,1,13,857,0.257143,0.254310,25.572660


# Deciding on a threshold
After assiging scores to the customers, the next step calls for deciding a threshold which would divide the best customers from the rest. Since we have a limited budget of 1000$ to distribute amongst the customers, we need to decide how many will get a coupon.

For the purpose of our exercise, it is safe to assume a '30 percent' discount may be enticing. 

* We will find the mean of all transactions and calculate 30 percent from it. 
* We can then divide the budget of 1000$ by the value obtained above to get the number of coupons we will distribute. 

In [20]:
#obtaining 30% of the mean transaction amount
coupon = data["tran_amount"].mean()*0.3
print(f'Coupon amount = {coupon}')
#dividing the budget by it
nr_of_customers = 1000/coupon
print(f'Number of Customers = {nr_of_customers}')

Coupon amount = 19.4975736
Number of Customers = 51.28843314123969


We will round off the coupon amount to 20 - which will get us the number of customers to be 50.

In [21]:
#picking the top 50 customers with the best score
top_50_churned = best_churn.loc[best_churn["churned"] == 1].head(50)

top_50_churned.head()

,churned,nr_of_transactions,amount_spent,scaled_tran,scaled_amount,score
customer_id,,,,,,
FM1112,1,15,1012,0.314286,0.309986,31.213567
FM1116,1,13,857,0.257143,0.254310,25.572660
FM1117,1,17,1185,0.371429,0.372126,37.177750
FM1123,1,19,1331,0.428571,0.424569,42.657020
FM1124,1,18,1127,0.400000,0.351293,37.564655
